In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
data = pd.read_parquet('data/reviews.pq')

In [3]:
data.head()

,userName,review,title,isEdited,date,rating,developerResponse,app_name
0,"mov ax,1/int 21h",OVERALL If you’re tired of companies like face...,"Secure, Easy to Use",False,2021-07-24 13:51:42,4,None,telegram-messenger
1,girlwithLion,✩ edit: the gift subscription button is not wo...,it’s my favorite app and i recommend it to eve...,False,2022-08-23 03:17:01,5,None,telegram-messenger
2,Mr.. Fisher,"Hey, I’ve been using Telegram for over 10+ yea...",In desperate need of Group Video Chat,False,2021-02-18 15:00:42,3,None,telegram-messenger
3,Zakir Hosen,I am enjoying telegram as my main chatting app...,"Need Video calling feature, faster passcode lo...",False,2019-09-20 15:54:38,4,"{'id': 20383789, 'body': 'Video calls are avai...",telegram-messenger
4,Mr Bcubed,The genuine telegram app deletes text messages...,Deleted messages,False,2020-05-30 05:14:18,5,None,telegram-messenger


In [4]:
def clean_reviews(text):
    res = text.replace('\n', ' ')
    # next I'll remove all non latin characters
    res = re.sub(r'[^\x00-\x7F\x80-\xFF\u0100-\u017F\u0180-\u024F\u1E00-\u1EFF]', u'', res)
    return ' '.join(res.split())

In [5]:
DROP = ['title', 'isEdited', 'developerResponse']

data = data.drop(DROP, axis=1)
data = data.drop_duplicates()
data['date'] = pd.to_datetime(data['date'])
data['rating'] = data['rating'].astype(int)
data['review'] = data['review'].apply(clean_reviews)

In [6]:
data.head()

,userName,review,date,rating,app_name
0,"mov ax,1/int 21h",OVERALL If youre tired of companies like faceb...,2021-07-24 13:51:42,4,telegram-messenger
1,girlwithLion,edit: the gift subscription button is not work...,2022-08-23 03:17:01,5,telegram-messenger
2,Mr.. Fisher,"Hey, Ive been using Telegram for over 10+ year...",2021-02-18 15:00:42,3,telegram-messenger
3,Zakir Hosen,I am enjoying telegram as my main chatting app...,2019-09-20 15:54:38,4,telegram-messenger
4,Mr Bcubed,The genuine telegram app deletes text messages...,2020-05-30 05:14:18,5,telegram-messenger


In [7]:
data = data.rename(columns={'userName': 'user_name'})

Other preprocessing activities will be done in next sections

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6300 entries, 0 to 6299
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   user_name  6300 non-null   object        
 1   review     6300 non-null   object        
 2   date       6300 non-null   datetime64[ns]
 3   rating     6300 non-null   int64         
 4   app_name   6300 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 295.3+ KB


In [9]:
data.describe()

,rating
count,6300.000000
mean,3.391746
std,1.554185
min,1.000000
25%,2.000000
50%,4.000000
75%,5.000000
max,5.000000


In [10]:
data.to_parquet('data/reviews_cleaned.pq', compression='GZIP')